In [2]:
import cloudscraper
from bs4 import BeautifulSoup
import boto3
import json
import pandas as pd
import csv


In [ ]:
with open('page.html', 'r', encoding='utf-8') as f:
    a = f.read()


In [ ]:
a

'<!DOCTYPE html><html class="no-js" lang="en-US"><!--<![endif]--><head>\n<title>Attention Required! | Cloudflare</title>\n<meta charset="UTF-8">\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n<meta http-equiv="X-UA-Compatible" content="IE=Edge">\n<meta name="robots" content="noindex, nofollow">\n<meta name="viewport" content="width=device-width,initial-scale=1">\n<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/cf.errors.css">\n<!--[if lt IE 9]><link rel="stylesheet" id=\'cf_styles-ie-css\' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->\n<style>body{margin:0;padding:0}</style>\n\n\n<!--[if gte IE 10]><!-->\n<script>\n  if (!navigator.cookieEnabled) {\n    window.addEventListener(\'DOMContentLoaded\', function () {\n      var cookieEl = document.getElementById(\'cookie-alert\');\n      cookieEl.style.display = \'block\';\n    })\n  }\n</script>\n<!--<![endif]-->\n\n\n</head>\n<body>\n  <div id="cf-wrapper">\n    <div class="cf-alert cf-alert

In [ ]:
soup = BeautifulSoup(a,'html.parser')


In [ ]:
soup.select("section.MuiCard-root[id^='stock-no-']")

[]

# scraping purpose

In [4]:
df = pd.read_csv('testing.csv')

In [5]:
# sqs = boto3.client('sqs', region_name='us-east-1')
sqs = boto3.client(
    'sqs',
    region_name='us-east-1',
    aws_access_key_id='test',
    aws_secret_access_key='test',
    endpoint_url='http://localhost:4566'
)
QUEUE_URL = 'http://sqs.us-east-1.localhost.localstack.cloud:4566/000000000000/webscraping'

In [54]:
attrs = sqs.get_queue_attributes(
    QueueUrl=QUEUE_URL,
    AttributeNames=['ApproximateNumberOfMessages']
)
num_messages = attrs['Attributes'].get('ApproximateNumberOfMessages', '0')
print(f"Approximate number of messages in queue: {num_messages}")

Approximate number of messages in queue: 0


In [7]:
def send_csv_to_queue(csv_file_path):
    with open(csv_file_path, mode='r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            message_body = json.dumps(row)
            response = sqs.send_message(QueueUrl=QUEUE_URL, MessageBody=message_body)
            # print(f"Sent message ID: {response['MessageId']}")

csv_path = 'testing.csv'
send_csv_to_queue(csv_path)

In [57]:
sqs.receive_message(QueueUrl=QUEUE_URL,MaxNumberOfMessages=10,WaitTimeSeconds=10).get('Messages',[])

[]

# bucket studd

In [ ]:
import boto3
LOCALSTACK_ENDPOINT = "http://localhost:4566"
BUCKET_NAME = "rapidious-datalake"
FILE_PATH = "testing.csv"
S3_KEY = "esting.csv"


s3 = boto3.client(
    "s3",
    region_name="us-east-1",
    aws_access_key_id="test",       # default LocalStack creds
    aws_secret_access_key="test",
    endpoint_url=LOCALSTACK_ENDPOINT
)

with open(FILE_PATH, "rb") as f:
    s3.put_object(Bucket=BUCKET_NAME, Key=S3_KEY, Body=f)

print(f"Uploaded {FILE_PATH} to s3://{BUCKET_NAME}/{S3_KEY}")

Uploaded testing.csv to s3://rapidious-datalake/esting.csv
